<a href="https://colab.research.google.com/github/ag826/AIPI590_XAI_F25/blob/main/Assignment_Interpretable_ML_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](
https://github.com/ag826/AIPI590_XAI_F25/blob/main/Assignment_Interpretable_ML_II.ipynb)


# Assignment - Interpretable ML II
**Adil Keku Gazder**

**ag825@duke.edu**

**For AIPI 590 - XAI, Fall 2025**

**Duke University**

https://colab.research.google.com/drive/1VM6rt1xVf9_7DBfhbGrV51gLvrA1Eg0b


For a Wine dataset, we use physiochemical characteristics to try and model the grade of the wine. We use PDP, ICE and ALE plots and analyze the findings from these plots.

### **Code to connect with github**

Adapted from the starter code shared by Brinnae Bent, shared here: https://github.com/AIPI-590-XAI/Duke-AI-XAI/blob/main/templates/template.ipynb

### **Installing required packages**

In [1]:
pip install ucimlrepo

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import classification_report
from sklearn.metrics import (
    confusion_matrix,
    roc_curve,
    roc_auc_score,
    recall_score,
    accuracy_score,
    precision_score,
    f1_score,
    mean_squared_error,
    mean_absolute_error
)
from sklearn.preprocessing import LabelEncoder, StandardScaler


In [4]:
from ucimlrepo import fetch_ucirepo

wine_quality = fetch_ucirepo(id=186)
df = wine_quality.data.original

## **1. Exploratory Data Analysis**

Code to get a better understanding of the dataset that we are working with, including a descripitive overview, samples and null values present in each column.

We also do a tad bit of preprocessing, including grouping numeric and categoric (dtype objects) columns together, along with visualizing our outcome variable (Churn against categorical variables).

In [5]:
print("#####################################################")
print("--- Shape Info ---")
print("#####################################################")
print("")
print(df.shape)
print("")
print("#####################################################")
print("--- Initial Data Info ---")
print("#####################################################")
print("")
df.info()
print("")
print("#####################################################")
print("--- First 5 Rows of Data ---")
print("#####################################################")
print("")
print(df.head())
print("")
print("#####################################################")
print("--- Null values of each column ---")
print("#####################################################")
print("")
print(df.isnull().sum())
print("")
print("#####################################################")
print("--- Data types of each column ---")
print("#####################################################")
print("")
print(df.dtypes)


#####################################################
--- Shape Info ---
#####################################################

(6497, 13)

#####################################################
--- Initial Data Info ---
#####################################################

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed_acidity         6497 non-null   float64
 1   volatile_acidity      6497 non-null   float64
 2   citric_acid           6497 non-null   float64
 3   residual_sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free_sulfur_dioxide   6497 non-null   float64
 6   total_sulfur_dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  a